# Evaluator Module

In [1]:
#| default_exp evaluator

In [2]:
#| export
import os
import CodeCheckList
import pandas as pd

import CodeCheckList.utils as utils
from CodeCheckList.tokenizer import CodeTokenizer
from CodeCheckList.masker import Masker
from CodeCheckList.predictor import Predictor
from CodeCheckList.judge import Judge

import statistics
import textdistance

/home/svelascodimate/miniconda3/envs/code-check-list/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#| export
class Evaluator:
    """Evaluator Module to perform all AST Evaluations"""
    def __init__(self, checkpoint: str, language, gpu_available=False):
        os.environ["TOKENIZERS_PARALLELISM"] = "true"
        self.tokenizer = CodeTokenizer.from_pretrained(checkpoint, language)
        self.masker = Masker(self.tokenizer)
        self.predictor = Predictor.from_pretrained(checkpoint, self.tokenizer, gpu_available)
        self.judge = Judge(self.tokenizer)

    def __call__(self, test_set, number_of_predictions: int, masking_rate: float):
        results_dict = self.evaluate_test_set(test_set, number_of_predictions, masking_rate)
        results_dataframe = pd.DataFrame([], columns=[
            'ast_element', 'occurences', 'jaccard', 'sorensen_dice', 'levenshtein', 'jaccard_avg', 'sorensen_dice_avg', 'levenshtein_avg'])
        for result_index, result in enumerate(results_dict):
            results_dataframe.loc[len(results_dataframe.index)] = [self.tokenizer.node_types[result_index], result[0], tuple(tuple(l) for l in result[1]), tuple(tuple(l) for l in result[2]), tuple(tuple(l) for l in result[3]), tuple(result[4]), tuple(result[5]), tuple(result[6])]
        return results_dataframe
    
    def evaluate_test_set(self, test_set, number_of_predictions: int, masking_rate: float):
        results_dict = []
        for node_type in self.tokenizer.node_types:
            results_dict.append([0,                                           #ocurrences
                                [[] for i in range(0,number_of_predictions)], #jaccard per prediction
                                [[] for i in range(0,number_of_predictions)], #sorensen_dice per prediction
                                [[] for i in range(0,number_of_predictions)], #levenshtein per prediction
                                [0 for i in range(0,number_of_predictions)],  #avg jaccard per prediction
                                [0 for i in range(0,number_of_predictions)],  #avg sorensen_dice per prediction
                                [0 for i in range(0,number_of_predictions)],  #avg levenshtein per prediction
                                ])
        for sample_index, sample in enumerate(test_set):
            print('-------- evaluating sample:'+str(sample_index)+' --------')
            for node_type_idx, node_type in enumerate(self.tokenizer.node_types):
                node_type_results = self.evaluate_node_type_on_snippet(sample['whole_func_string'], node_type_idx, number_of_predictions, masking_rate)
                if(len(node_type_results)>0):
                    results_dict[node_type_idx][0] += node_type_results[0][0]
                    for prediction_number_index in range(0, number_of_predictions):
                        if(node_type_results[prediction_number_index][1]!=None):
                            results_dict[node_type_idx][1][prediction_number_index].append(node_type_results[prediction_number_index][1])
                            results_dict[node_type_idx][4][prediction_number_index] = round(statistics.mean(results_dict[node_type_idx][1][prediction_number_index]),3)
                        if(node_type_results[prediction_number_index][2]!=None):
                            results_dict[node_type_idx][2][prediction_number_index].append(node_type_results[prediction_number_index][2])
                            results_dict[node_type_idx][5][prediction_number_index] = round(statistics.mean(results_dict[node_type_idx][2][prediction_number_index]),3)
                        if(node_type_results[prediction_number_index][3]!=None):
                            results_dict[node_type_idx][3][prediction_number_index].append(node_type_results[prediction_number_index][3])
                            results_dict[node_type_idx][6][prediction_number_index] = round(statistics.mean(results_dict[node_type_idx][3][prediction_number_index]),3)
        return results_dict
        
    def evaluate_node_type_on_snippet(self, source_code: str, target_node_type_idx: int, number_of_predictions: int, masking_rate: float):
        results=[]
        
        source_code_tree = self.tokenizer.parser.parse(bytes(source_code, "utf8"))
        source_code_nodes = []
        utils.find_nodes(source_code_tree.root_node, self.tokenizer.node_types[target_node_type_idx], source_code_nodes)
        if len(source_code_nodes) == 0:
            return results

        masked_code_encoding = self.masker(source_code, self.tokenizer(source_code), target_node_type_idx, masking_rate)
        predictions = self.predictor(masked_code_encoding, self.tokenizer.tokenizer(source_code, return_tensors='pt'), number_of_predictions)  

        for prediction_number in range(0, number_of_predictions):
            predicted_code = predictions[prediction_number]
            prediction_results = self.judge(source_code, predicted_code)
            results.append([len(source_code_nodes), prediction_results[0], prediction_results[1], prediction_results[2]])
        return results


## Full Pipeline

### Download Grammar

In [4]:
from CodeCheckList import loader

"""define language"""
python_language = "python"

languages = [python_language]

loader.download_grammars(languages)

/home/svelascodimate/miniconda3/envs/code-check-list/lib/python3.10/site-packages/CodeCheckList/grammars


### Load Model

In [5]:

"""define the model checkpoint"""
checkpoint = "huggingface/CodeBERTa-small-v1"

### Create Modules

In [6]:
from CodeCheckList.tokenizer import CodeTokenizer
from CodeCheckList.masker import Masker

#create code tokenizer 
bert_tokenizer = CodeTokenizer.from_pretrained(checkpoint, python_language)

#create code masker
code_masker = Masker(bert_tokenizer)

### Node Types

In [7]:
print(bert_tokenizer.node_types)

['aliased_import', '@', 'relative_import', '|=', '**=', 'exec_statement', '|', 'augmented_assignment', 'lambda_parameters', 'def', '*', ')', 'import_prefix', '_simple_statement', 'tuple_pattern', 'false', '^', 'if_clause', '&', ',', 'as', 'nonlocal', 'yield', '%', 'format_specifier', '-', 'comment', 'not_operator', 'else_clause', 'keyword_argument', 'dictionary_comprehension', '-=', 'ERROR', 'await', '>>=', 'lambda', 'named_expression', 'argument_list', '/', '=', 'dictionary_splat_pattern', 'return_statement', 'decorated_definition', 'escape_sequence', 'class_definition', 'break_statement', 'nonlocal_statement', 'try_statement', 'expression_list', '<<=', '!=', '^=', 'not', 'try', 'pattern', 'as_pattern_target', 'raise_statement', '//=', 'del', 'finally_clause', 'with_statement', '.', 'for', 'integer', 'default_parameter', 'raise', 'none', 'and', '__future__', 'if_statement', 'print_statement', '<>', '>', 'global_statement', 'comparison_operator', 'unary_operator', '}}', 'return', 'func

### Encodings

In [8]:
"""example source code"""

code = "def multiply_numbers(a,b):\n    return a*b"
#code = "def scale(self, center=True, scale=True):\n        \"\"\"\nthe the\n\n\n                                                                                                                                                          _\n                     ____________=_=_===========________===______________________________==_____________________\n_______\n____\n\n___\n\n\n\n\n\n\n\n\n        return return)"
#code = "def hello_world(a,b):\n    print('hello world')"
#code = "def __ordered_values_by_indexes(self, data, inds): \"\"\" Return values (intensities) by indexes. Used for multiscale graph cut. data = [[0 1 1], [0 2 2], [0 2 2]] inds = [[0 1 2], [3 4 4], [5 4 4]] return: [0, 1, 1, 0, 2, 0] If the data are not consistent, it will take the maximal value \"\"\" # get unique labels and their first indexes # lab, linds = np.unique(inds, return_index=True) # compute values by indexes # values = data.reshape(-1)[linds] # alternative slow implementation # if there are different data on same index, it will take # maximal value # lab = np.unique(inds) # values = [0]*len(lab) # for label in lab: # values[label] = np.max(data[inds == label]) # # values = np.asarray(values) # yet another implementation values = [None] * (np.max(inds) + 1) linear_inds = inds.ravel() linear_data = data.ravel() for i in range(0, len(linear_inds)): # going over all data pixels if values[linear_inds[i]] is None: # this index is found for first values[linear_inds[i]] = linear_data[i] elif values[linear_inds[i]] < linear_data[i]: # here can be changed maximal or minimal value values[linear_inds[i]] = linear_data[i] values = np.asarray(values) return values"
#code = "def __ordered_values_by_indexes(self, data, inds):  # get unique labels and their first indexes # lab, linds = np.unique(inds, return_index=True) # compute values by indexes # values = data.reshape(-1)[linds] # alternative slow implementation # if there are different data on same index, it will take # maximal value # lab = np.unique(inds) # values = [0]*len(lab) # for label in lab: # values[label] = np.max(data[inds == label]) # # values = np.asarray(values) # yet another implementation values = [None] * (np.max(inds) + 1) linear_inds = inds.ravel() linear_data = data.ravel() for i in range(0, len(linear_inds)): # going over all data pixels if values[linear_inds[i]] is None: # this index is found for first values[linear_inds[i]] = linear_data[i] elif values[linear_inds[i]] < linear_data[i]: # here can be changed maximal or minimal value values[linear_inds[i]] = linear_data[i] values = np.asarray(values) return values"
target_node_type = "identifier"

#encoding 
source_code_encoding = bert_tokenizer(code)

#masking
masked_code_encoding = code_masker(code, bert_tokenizer(code), bert_tokenizer.node_types.index(target_node_type), 1)

assert len(source_code_encoding['input_ids']) == len(masked_code_encoding['input_ids'])

#masked code
masked_code = bert_tokenizer.tokenizer.decode(list(filter(lambda token_id: False if token_id == bert_tokenizer.tokenizer.bos_token_id or 
            token_id == bert_tokenizer.tokenizer.eos_token_id else True, masked_code_encoding['input_ids'])))

print(masked_code)

def<mask><mask><mask>(<mask>,<mask>):
    return<mask>*<mask>


### Code Prediction

In [9]:
from CodeCheckList.predictor import Predictor

predictor = Predictor.from_pretrained(checkpoint, bert_tokenizer)
predictions = predictor(masked_code_encoding, bert_tokenizer.tokenizer(code, return_tensors='pt'), 5)

### Evaluation

In [10]:
import CodeCheckList.utils as utils

prediction_number = 0
print('------------- CODE -------------')
print(code)
print('\n---------- MASKED -------------')
print(masked_code)
print('\n--------- PREDICTED -----------')
predicted_code = predictions[prediction_number]
print(predicted_code)
print('\n--------- AST COMPARE -----------')
filtered_nodes = []
filtered_nodes_predict = []
utils.find_nodes(bert_tokenizer.parser.parse(bytes(code, "utf8")).root_node, bert_tokenizer.node_types[bert_tokenizer.node_types.index(target_node_type)], filtered_nodes)
utils.find_nodes(bert_tokenizer.parser.parse(bytes(predicted_code, "utf8")).root_node, bert_tokenizer.node_types[bert_tokenizer.node_types.index(target_node_type)], filtered_nodes_predict)
print(len(filtered_nodes))
print(len(filtered_nodes_predict))
#base the evaluation on size comparison

------------- CODE -------------
def multiply_numbers(a,b):
    return a*b

---------- MASKED -------------
def<mask><mask><mask>(<mask>,<mask>):
    return<mask>*<mask>

--------- PREDICTED -----------
def __function(name, value):
    return f*args

--------- AST COMPARE -----------
5
5


## Testing

In [11]:
from datasets import load_dataset 
import CodeCheckList.utils as utils


evaluator = Evaluator(checkpoint, python_language)

max_token_number = bert_tokenizer.tokenizer.max_len_single_sentence
print(max_token_number)

test_set = load_dataset("code_search_net", split='test')
test_set = utils.get_test_sets(test_set, python_language, max_token_number, bert_tokenizer)

510


No config specified, defaulting to: code_search_net/all
Found cached dataset code_search_net (/home/svelascodimate/.cache/huggingface/datasets/code_search_net/all/1.0.0/80a244ab541c6b2125350b764dc5c2b715f65f00de7a56107a28915fac173a27)
Parameter 'function'=<function get_test_sets.<locals>.<lambda> at 0x7f118f2636d0> of the transform datasets.arrow_dataset.Dataset.filter@2.0.1 couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 101/101 [00:17<00:00,  5.92ba/s]


In [12]:
print(test_set[0]['whole_func_string'])

def get_vid_from_url(url):
        """Extracts video ID from URL.
        """
        return match1(url, r'youtu\.be/([^?/]+)') or \
          match1(url, r'youtube\.com/embed/([^/?]+)') or \
          match1(url, r'youtube\.com/v/([^/?]+)') or \
          match1(url, r'youtube\.com/watch/([^/?]+)') or \
          parse_query_param(url, 'v') or \
          parse_query_param(parse_query_param(url, 'u'), 'v')


In [13]:
print('TOTAL PYTHON FILTERED SAMPLES: '+str(len(test_set)))
test_set = utils.get_random_sub_set_test_set(utils.get_test_sets(load_dataset("code_search_net", split='test'), "python", evaluator.tokenizer.tokenizer.max_len_single_sentence, evaluator.tokenizer), 5)
print('TOTAL SAMPLES TO EVALUATE: '+str(len(test_set)))
len(test_set)

TOTAL PYTHON FILTERED SAMPLES: 19408


No config specified, defaulting to: code_search_net/all
Found cached dataset code_search_net (/home/svelascodimate/.cache/huggingface/datasets/code_search_net/all/1.0.0/80a244ab541c6b2125350b764dc5c2b715f65f00de7a56107a28915fac173a27)
100%|██████████| 101/101 [00:17<00:00,  5.83ba/s]


TOTAL SAMPLES TO EVALUATE: 5


5

In [14]:
number_of_predictions = 3
checkpoint = "huggingface/CodeBERTa-small-v1"
python_language = "python"
masking_rate = 1

evaluator = Evaluator(checkpoint, python_language, gpu_available=False)

results_dataframe = evaluator(test_set, number_of_predictions, masking_rate)

results_dataframe.sort_values(by=['occurences'], ascending=False)


-------- evaluating sample:0 --------
-------- evaluating sample:1 --------
-------- evaluating sample:2 --------
-------- evaluating sample:3 --------
-------- evaluating sample:4 --------


,ast_element,occurences,jaccard,sorensen_dice,levenshtein,jaccard_avg,sorensen_dice_avg,levenshtein_avg
165,identifier,116,"((1.0, 0.9166666666666666, 0.9010989010989011,...","((1.0, 0.9565217391304348, 0.9479768786127167,...","((1.0, 0.9166666666666666, 0.8901098901098901,...","(0.923, 0.873, 0.817)","(0.959, 0.93, 0.894)","(0.913, 0.864, 0.817)"
85,"""",32,"((1.0, 1.0, 1.0, 0.9097222222222222, 1.0), (1....","((1.0, 1.0, 1.0, 0.9527272727272728, 1.0), (1....","((1.0, 1.0, 1.0, 0.9420289855072463, 1.0), (1....","(0.982, 0.958, 0.555)","(0.991, 0.977, 0.681)","(0.988, 0.959, 0.554)"
19,",",28,"((1.0, 1.0, 1.0, 1.0, 1.0), (0.775, 0.97058823...","((1.0, 1.0, 1.0, 1.0, 1.0), (0.873239436619718...","((1.0, 1.0, 1.0, 1.0, 1.0), (0.789473684210526...","(1.0, 0.834, 0.85)","(1.0, 0.899, 0.913)","(1.0, 0.848, 0.851)"
168,expression_statement,23,"((0.23880597014925375, 0.9420289855072463, 0.5...","((0.3855421686746988, 0.9701492537313433, 0.73...","((0.24242424242424243, 0.9338235294117647, 0.5...","(0.581, 0.468, 0.57)","(0.709, 0.611, 0.724)","(0.575, 0.417, 0.519)"
130,(,19,"((1.0, 1.0, 1.0, 1.0, 1.0), (0.783783783783783...","((1.0, 1.0, 1.0, 1.0, 1.0), (0.878787878787878...","((1.0, 1.0, 1.0, 1.0, 1.0), (0.787878787878787...","(1.0, 0.876, 0.955)","(1.0, 0.932, 0.976)","(1.0, 0.87, 0.96)"
...,...,...,...,...,...,...,...,...
80,pass_statement,0,"((), (), ())","((), (), ())","((), (), ())","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)"
81,list_pattern,0,"((), (), ())","((), (), ())","((), (), ())","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)"
82,<,0,"((), (), ())","((), (), ())","((), (), ())","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)"
86,typed_parameter,0,"((), (), ())","((), (), ())","((), (), ())","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)"
